# BAB 10: Language Modeling dengan TensorFlow

## Ringkasan

Bab ini membahas **language modeling**, yaitu tugas memprediksi kata berikutnya berdasarkan sequence kata sebelumnya. Ini adalah dasar dari model NLP modern seperti BERT dan GPT. Model yang dibangun akan menggunakan dataset cerita anak-anak bAbI, dengan teknik n-gram untuk mengurangi vocabulary size, GRU sebagai arsitektur model, dan beam search untuk meningkatkan kualitas teks yang dihasilkan.

---

## Konsep Utama

### Language Modeling: Definisi dan Aplikasi

Language modeling menghitung probabilitas P(w_n | w₁, w₂, ..., w_{n-1}) - yaitu, berapa kemungkinan kata w_n muncul setelah sequence kata sebelumnya. Contoh praktis adalah saat Anda menulis pesan di smartphone dan keyboard menyarankan kata berikutnya. Ini adalah supervised learning task tanpa label eksplisit - kita hanya perlu melatih model untuk memprediksi next word dari previous words.

### Markov Property

Menghitung probabilitas dengan melihat seluruh history computationally infeasible untuk teks panjang. Markov Property menawarkan solusi: kita bisa mengaproksimasi probabilitas dengan hanya melihat k kata terakhir, bukan seluruh history. Formula: P(w_n | w₁, w₂, ..., w_{n-1}) ≈ P(w_n | w_k, w_{k+1}, ..., w_{n-1}). Ini adalah trade-off antara akurasi dan efisiensi komputasi.

### N-Grams: Mengatasi Vocabulary Besar

Dataset bAbI memiliki 14,473 unique words (yang muncul minimal 10 kali). Ini membuat final softmax layer sangat besar dan computationally expensive. **N-grams** mengatasi ini dengan memecah teks menjadi sub-unit karakter dengan panjang tetap n.

Contoh bigrams dari "I went to":
- "I ", " w", "we", "en", "nt", "t ", " t", "to", "o "

Dengan menggunakan bigrams, vocabulary size turun drastis dari 14,473 menjadi hanya 735! Keuntungan n-grams:
1. **Vocabulary size reduction** - Signifikan mengurangi parameters
2. **Fewer OOV words** - Kata baru bisa dikonstruksi dari n-grams yang dikenal
3. **Better generalization** - Model belajar pola di level granular

---

## Dataset dan Preprocessing

### Dataset bAbI

bAbI adalah dataset cerita anak-anak dari Facebook Research yang ideal untuk language modeling karena:
- Bahasa sederhana dan tidak kompleks
- Struktur naratif yang jelas
- Vocabulary terbatas dan repetitif
- Dataset berisi 98 training stories, 5 validation stories, 5 test stories

Setiap cerita diawali dengan marker "_BOOK_TITLE_" yang memungkinkan pemisahan cerita individual.

### Tokenization

Setelah mengkonversi text menjadi n-grams, kita perlu mengubahnya menjadi numerical IDs karena neural networks hanya memproses numbers. **Keras Tokenizer** melakukan ini dalam dua fase:

1. **Fitting phase**: Membaca semua training data dan membangun vocabulary dictionary yang memetakan n-gram ke integer ID
2. **Transformation phase**: Menggunakan dictionary untuk mengkonversi sequences of n-grams menjadi sequences of IDs

Penting: Tokenizer hanya di-fit pada training data untuk menghindari data leakage.

### TensorFlow Data Pipeline

Pipeline mengubah sequence panjang arbitrary menjadi fixed-size windows yang bisa diproses neural network. Prosesnya:

1. Convert text ke sequences of IDs
2. Gunakan `tf.data.Dataset.window()` untuk membuat windowed sequences
3. Gunakan `flat_map()` untuk menghilangkan nested structure
4. Batch data
5. Split menjadi input-target pairs (target = input shifted right by 1)

Alasan menggunakan window size `n_seq + 1`: kita butuh extra element agar target dapat digenerate dengan benar (input dan target harus same length).

---

## GRU: Gated Recurrent Unit

### Perbedaan LSTM dan GRU

**GRU** adalah simplifikasi dari LSTM yang tetap mempertahankan performance. Perbedaan utama:

| Aspek | LSTM | GRU |
|-------|------|-----|
| **States** | Dual (cell state, output state) | Single (hidden state) |
| **Gates** | 3 gates (input, forget, output) | 2 gates (update, reset) |
| **Complexity** | Lebih kompleks, training lebih lambat | Lebih sederhana, training lebih cepat |
| **Performance** | Sedikit lebih baik untuk long dependencies | Comparable performance dengan LSTM |

### GRU Mechanics

GRU memiliki dua gates dan mengikuti persamaan ini:

1. **Reset gate**: \( r_t = σ(W_{rh}h_{t-1} + W_{rx}x_t + b_r) \)
   - Mengontrol berapa banyak previous state yang di-reset
   - Nilai 0 = reset semuanya, 1 = pertahankan semuanya

2. **Update gate**: \( z_t = σ(W_{zh}h_{t-1} + W_{zx}x_t + b_z) \)
   - Mengontrol balance antara previous state dan new information

3. **New state candidate**: \( \tilde{h}_t = tanh(W_h(r_t ⊙ h_{t-1}) + W_xx_t + b) \)
   - Compute candidate untuk new state
   - Perhatikan perkalian elemen-wise dengan reset gate

4. **Final state**: \( h_t = z_t ⊙ h_{t-1} + (1 - z_t) ⊙ \tilde{h}_t \)
   - Weighted average antara previous state dan new candidate
   - Controlled by update gate

GRU cocok untuk language modeling karena: proses sequential, dapat handle variable-length context, dan computational efficiency.

---

## Evaluasi Model: Perplexity

### Apa itu Perplexity?

**Perplexity** adalah metric untuk mengukur seberapa baik language model. Secara intuitif, perplexity mengukur "berapakah jumlah equally likely choices yang model pikir ada untuk next word."

Perplexity dihitung sebagai exponential dari cross-entropy loss:
\[ Perplexity = e^{H(X)} \]

Di mana H(X) adalah entropy (uncertainty) dari distribusi probabilitas model.

### Intuisi Perplexity

- **Perplexity rendah** (misal, 2): Model sangat confident, hanya 2 equally likely candidates
- **Perplexity tinggi** (misal, 1000): Model tidak confident, banyak equally likely candidates

**Contoh**: Untuk kalimat "I went swimming in the ____":
- Model yang prediksi {pool: 0.8, sea: 0.15, lake: 0.05} → Perplexity ≈ 1.5 (confident)
- Model yang prediksi {10 words, each 0.1} → Perplexity ≈ 10 (uncertain)

Model pertama lebih baik - lebih percaya diri dan fokus pada pilihan yang masuk akal.

### Perplexity vs Accuracy

Mengapa tidak gunakan accuracy saja? Karena accuracy terlalu strict - jika true word adalah "dog" tapi model prediksi "cat", accuracy = 0%, padahal "cat" dan "dog" semantically similar. Perplexity lebih nuanced - bahkan jika tidak exactly correct, jika model assign reasonable probability ke similar words, perplexity tetap moderate.

---

## Text Generation Strategies

### Greedy Decoding

**Greedy decoding** adalah approach paling sederhana: di setiap timestep, pilih word dengan probabilitas tertinggi sebagai next word, gunakan sebagai input untuk timestep berikutnya.

**Kelemahan:**
1. **Repetitive text** - Model bisa stuck dalam loops ("said the king said the king...")
2. **Myopic** - Hanya melihat satu step ahead, tidak bisa long-term planning
3. **No exploration** - Tidak explore alternative paths yang mungkin lebih baik

Mitigasi sederhana: Occasionally random pilih dari top-3 predictions instead of top-1 untuk break repetitive patterns.

### Beam Search

**Beam search** adalah strategi yang jauh lebih sophisticated. Daripada commit ke single best choice setiap step, beam search explore multiple promising paths simultaneously.

**Parameters:**
- **Beam width (k)**: Number of top candidates to explore (misal, k=3)
- **Beam depth (d)**: How many steps ahead to look (misal, d=5)

**How it works:**

1. **Timestep 1**: Dari semua vocabulary, keep top-3 candidates
2. **Timestep 2**: Untuk setiap dari 3 candidates, generate top-3 next words → 9 total candidates
3. **Pruning**: Keep hanya top-3 dari 9 berdasarkan joint probability
4. **Repeat**: Continue untuk d timesteps
5. **Final selection**: Choose sequence dengan highest joint probability

**Keuntungan:**
- Explore multiple paths, tidak stuck di single path
- Better long-term planning
- Empirically menghasilkan teks lebih coherent dan less repetitive
- Tradeof: Computational cost lebih tinggi (k^d possibilities explored)

**Diverse beam search**: Enhancement dari standard beam search dengan menambahkan diversity term untuk encourage truly different paths, bukan hanya variations dari same idea.

---

## Program-Program Implementasi

### Program 1: Download dan Extract Dataset

```python
import os
import requests
import tarfile
import shutil

# Cek apakah file sudah ada di local disk
if not os.path.exists(os.path.join('data', 'lm','CBTest.tgz')):
    
    # Download dari server
    url = "http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz"
    r = requests.get(url)
    
    # Buat direktori jika belum ada
    if not os.path.exists(os.path.join('data','lm')):
        os.makedirs(os.path.join('data','lm'), exist_ok=True)
    
    # Write binary file
    with open(os.path.join('data', 'lm', 'CBTest.tgz'), 'wb') as f:
        f.write(r.content)
    print("✓ File downloaded")
else:
    print("✓ File already exists, skip download")

# Extract compressed archive
if not os.path.exists(os.path.join('data', 'lm', 'CBTest')):
    with tarfile.open(os.path.join("data","lm","CBTest.tgz"), 'r') as tarf:
        tarf.extractall(os.path.join("data","lm"))
    print("✓ File extracted")
else:
    print("✓ File already extracted")
```

**Penjelasan**: Program ini download dataset dari server (dengan caching - jika sudah ada, tidak download lagi) dan mengextract compressed archive. Ini adalah idempotent - running multiple times tidak ada masalah.

---

### Program 2: Membaca Stories

```python
def read_data(path):
    """
    Baca stories dari text file.
    Dataset structure: setiap story diawali dengan "_BOOK_TITLE_"
    """
    stories = []     # Hold all completed stories
    s = []           # Hold lines dari current story
    
    with open(path, 'r') as f:
        for row in f:
            # Cek jika ini marker untuk story baru
            if row.startswith("_BOOK_TITLE_"):
                # Jika ada story yang sudah terakumulasi, add ke list
                if len(s) > 0:
                    stories.append(' '.join(s).lower())
                # Reset untuk story baru
                s = []
            # Append baris saat ini ke current story
            s.append(row)
    
    # Handle edge case: final story belum ditambahkan
    if len(s) > 0:
        stories.append(' '.join(s).lower())
    
    return stories

# Load datasets
stories = read_data(os.path.join('data','lm','CBTest','data','cbt_train.txt'))
val_stories = read_data(os.path.join('data','lm','CBTest','data','cbt_valid.txt'))
test_stories = read_data(os.path.join('data','lm','CBTest','data','cbt_test.txt'))

print(f"Loaded {len(stories)} training stories")
```

**Penjelasan**: Function ini menggunakan state machine pattern untuk parse file structure. Setiap kali ketemu "_BOOK_TITLE_", program close previous story dan start new one. Ini handle edge case untuk final story yang belum diproses.

---

### Program 3: Generate N-Grams

```python
def get_ngrams(text, n):
    """
    Generate n-grams dari text
    Menggunakan non-overlapping approach (stride = n)
    """
    return [text[i:i+n] for i in range(0, len(text), n)]

# Test
test_string = "I like chocolates"
print("Original:", test_string)
for i in range(1, 4):
    print(f"{i}-grams: {get_ngrams(test_string, i)}")

# Analyze vocabulary size dengan bigrams
from collections import Counter
from itertools import chain

ngrams = 2
text = chain(*[get_ngrams(s, ngrams) for s in stories])
cnt = Counter(text)

freq_df = pd.Series(list(cnt.values()), index=list(cnt.keys())).sort_values(ascending=False)

n_vocab = (freq_df >= 10).sum()
print(f"Vocabulary size (frequency >= 10): {n_vocab}")
# Output: ~735 (vs 14,473 words - huge reduction!)
```

**Penjelasan**: N-grams diproduksi dengan sliding window. Program ini menunjukkan bagaimana bigrams sangat reduce vocabulary size dari ~14K words menjadi ~735 n-grams. Ini adalah massive reduction yang membuat model lebih efficient.

---

### Program 4: Keras Tokenizer

```python
from tensorflow.keras.preprocessing.text import Tokenizer

# Generate n-grams dari semua training stories
train_ngram_stories = [get_ngrams(s, 2) for s in stories]

# Create tokenizer
tokenizer = Tokenizer(num_words=n_vocab, oov_token='unk', lower=False)

# Fit HANYA pada training data (penting: avoid data leakage)
tokenizer.fit_on_texts(train_ngram_stories)

# Convert text ke sequences of IDs
train_data_seq = tokenizer.texts_to_sequences(train_ngram_stories)
val_ngram_stories = [get_ngrams(s, 2) for s in val_stories]
val_data_seq = tokenizer.texts_to_sequences(val_ngram_stories)
test_ngram_stories = [get_ngrams(s, 2) for s in test_stories]
test_data_seq = tokenizer.texts_to_sequences(test_ngram_stories)

# Check mapping
print(f"Bigram 'th' -> ID {tokenizer.word_index.get('th')}")
print(f"ID 2 -> Bigram '{tokenizer.index_word.get(2)}'")

# Example: original vs tokenized
print("\nOriginal bigrams:", test_ngram_stories[0][:10])
print("Tokenized IDs:", test_data_seq[0][:10])
```

**Penjelasan**: Tokenizer build dictionary yang map setiap n-gram ke unique integer ID. Penting fit HANYA pada training data untuk avoid data leakage. Output adalah sequences of IDs yang siap diproses neural network.

---

### Program 5: TensorFlow Data Pipeline

```python
import tensorflow as tf

def get_tf_pipeline(data_seq, n_seq, batch_size=64, shift=1, shuffle=True):
    """
    Buat tf.data pipeline yang convert sequences ke fixed-length windows.
    Window size = n_seq + 1 (extra 1 untuk generate target)
    """
    
    # Create dataset dari ragged tensor (variable-length sequences)
    text_ds = tf.data.Dataset.from_tensor_slices(tf.ragged.constant(data_seq))
    
    # Shuffle di level story
    if shuffle:
        text_ds = text_ds.shuffle(buffer_size=len(data_seq)//2)
    
    # Flat map untuk window creation
    # This adalah complex nested operation - dijelaskan detail di atas
    text_ds = text_ds.flat_map(
        lambda x: tf.data.Dataset.from_tensor_slices(x).window(
            n_seq+1, shift=shift
        ).flat_map(
            lambda window: window.batch(n_seq+1, drop_remainder=True)
        )
    )
    
    # Shuffle di level window
    if shuffle:
        text_ds = text_ds.shuffle(buffer_size=10*batch_size)
    
    # Batch data
    text_ds = text_ds.batch(batch_size)
    
    # Split setiap sequence ke input dan target
    # Target = input shifted right by 1
    text_ds = text_ds.map(lambda x: (x[:,:-1], x[:, 1:])).prefetch(
        buffer_size=tf.data.experimental.AUTOTUNE)
    
    return text_ds

# Create pipelines
batch_size = 64
train_ds = get_tf_pipeline(train_data_seq, n_seq=100, batch_size=batch_size, shuffle=True)
valid_ds = get_tf_pipeline(val_data_seq, n_seq=100, batch_size=batch_size, shuffle=False)
test_ds = get_tf_pipeline(test_data_seq, n_seq=100, batch_size=batch_size, shuffle=False)

# Verify pipeline output
for inputs, targets in train_ds.take(1):
    print(f"Input shape: {inputs.shape}")
    print(f"Target shape: {targets.shape}")
    print(f"Sample input: {inputs[0][:5]}")
    print(f"Sample target: {targets[0][:5]}")
```

**Penjelasan**: Pipeline menggunakan windowing untuk membuat fixed-size sequences dari variable-length stories. Proses ini complex karena nested operations, tapi hasil akhirnya adalah batch dari (input, target) pairs di mana target adalah input shifted right by 1.

---

### Program 6: GRU Model Architecture

```python
import tensorflow as tf

# Model definition
model = tf.keras.models.Sequential([
    # Embedding layer - convert IDs ke dense vectors
    tf.keras.layers.Embedding(
        input_dim=n_vocab,      # Vocabulary size
        output_dim=128,         # Embedding dimension
        mask_zero=True,         # Mask padding zeros
        input_shape=(n_seq,)
    ),
    
    # GRU layer - process sequence
    tf.keras.layers.GRU(
        units=1024,             # Hidden state dimension
        return_sequences=True   # Return all timesteps
    ),
    
    # Dense layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Output layer - predict next n-gram ID
    tf.keras.layers.Dense(n_vocab)  # No activation (raw logits)
])

# Compile model
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=[PerplexityMetric()]  # Custom metric
)

model.summary()
```

**Penjelasan**: Model menggunakan Embedding layer untuk convert IDs ke dense vectors, GRU untuk process sequences, dan Dense output layer untuk predict next n-gram ID. Loss adalah SparseCategoricalCrossentropy karena target adalah single integer ID, bukan one-hot vector.

---

### Program 7: Custom Perplexity Metric

```python
class PerplexityMetric(tf.keras.metrics.Mean):
    """Custom metric untuk menghitung perplexity"""
    
    def __init__(self, name='perplexity', **kwargs):
        super().__init__(name=name, **kwargs)
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        # Compute cross-entropy loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.cast(tf.reshape(y_true, [-1]), tf.int32),
            logits=tf.reshape(y_pred, [-1, n_vocab])
        )
        
        # Compute perplexity = exp(loss)
        perplexity = tf.exp(loss)
        
        # Update metric
        super().update_state(perplexity, sample_weight)

# Usage
metrics = [PerplexityMetric()]
```

**Penjelasan**: Custom metric menghitung perplexity sebagai exponential dari cross-entropy loss. Lower perplexity = better model.

---

### Program 8: Training dengan Callbacks

```python
# Define callbacks
callbacks = [
    # Early stopping jika validation perplexity tidak improve
    tf.keras.callbacks.EarlyStopping(
        monitor='val_perplexity',
        mode='min',
        patience=5,
        verbose=1
    ),
    
    # Reduce learning rate jika validation perplexity plateau
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_perplexity',
        mode='min',
        factor=0.5,
        patience=3,
        verbose=1
    ),
    
    # Save best model
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_perplexity',
        mode='min',
        save_best_only=True,
        verbose=1
    )
]

# Train model
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=20,
    callbacks=callbacks,
    verbose=1
)
```

**Penjelasan**: Training dengan callbacks untuk early stopping, learning rate reduction, dan model checkpointing. Ini standard practice untuk prevent overfitting dan save best model.

---

### Program 9: Greedy Text Generation

```python
def generate_text_greedy(model, seed_text, length=100, temperature=1.0):
    """
    Generate text menggunakan greedy decoding dengan randomness.
    
    Args:
        model: Trained model
        seed_text: Initial text (string)
        length: Berapa banyak bigrams untuk generate
        temperature: Control randomness (1.0 = use model's distribution)
    """
    
    # Convert seed text to n-grams
    seed_ngrams = get_ngrams(seed_text, 2)
    input_seq = tokenizer.texts_to_sequences([seed_ngrams])[0]
    
    # Pad ke n_seq length jika perlu
    input_seq = tf.keras.preprocessing.sequence.pad_sequences(
        [input_seq], maxlen=n_seq, padding='pre')[0]
    
    # Generate loop
    generated_ngrams = seed_ngrams.copy()
    
    for _ in range(length):
        # Prepare input
        current_input = tf.reshape(input_seq[-n_seq:], [1, n_seq])
        
        # Predict logits
        logits = model.predict(current_input, verbose=0)[0, -1, :]
        
        # Apply temperature
        logits = logits / temperature
        
        # Get probabilities
        probs = tf.nn.softmax(logits).numpy()
        
        # Sample next ID
        # Greedy: always pick max
        next_id = np.argmax(probs)
        
        # Optional: Add randomness - sometimes pick from top-3
        if np.random.rand() < 0.1:  # 10% chance
            top_3_ids = np.argsort(probs)[-3:]
            next_id = np.random.choice(top_3_ids)
        
        # Convert ID back to n-gram
        next_ngram = tokenizer.index_word.get(next_id, 'unk')
        generated_ngrams.append(next_ngram)
        
        # Update input for next iteration
        input_seq = np.append(input_seq[1:], next_id)
    
    # Join n-grams to form text
    generated_text = ''.join(generated_ngrams)
    return generated_text

# Generate example
seed = "once upon a time"
generated = generate_text_greedy(model, seed, length=200)
print(generated)
```

**Penjelasan**: Greedy decoding selalu pick word dengan highest probability. Kami tambah optional randomness - 10% waktu pick dari top-3 untuk avoid repetition. Temperature parameter control randomness dari distribution.

---

### Program 10: Beam Search Text Generation

```python
def generate_text_beam_search(model, seed_text, length=50, beam_width=3):
    """
    Generate text menggunakan beam search.
    
    Args:
        model: Trained model
        seed_text: Initial text
        length: Berapa banyak bigrams untuk generate
        beam_width: Berapa candidate paths untuk track
    """
    
    # Initialize seed
    seed_ngrams = get_ngrams(seed_text, 2)
    input_seq = tokenizer.texts_to_sequences([seed_ngrams])[0]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences(
        [input_seq], maxlen=n_seq, padding='pre')[0]
    
    # Beam search candidates: list of (sequence, log_probability)
    candidates = [(input_seq.copy(), 0.0)]
    
    # Generate loop
    for step in range(length):
        next_candidates = []
        
        # For each current candidate
        for current_seq, current_log_prob in candidates:
            
            # Get model predictions
            current_input = tf.reshape(current_seq[-n_seq:], [1, n_seq])
            logits = model.predict(current_input, verbose=0)[0, -1, :]
            probs = tf.nn.softmax(logits).numpy()
            
            # Get top beam_width candidates
            top_ids = np.argsort(probs)[-beam_width:]
            
            # For each top candidate
            for next_id in top_ids:
                # Calculate log probability (more stable than probability)
                log_prob = np.log(probs[next_id])
                new_log_prob = current_log_prob + log_prob
                
                # Create new sequence
                new_seq = np.append(current_seq[1:], next_id)
                
                # Add to candidates
                next_candidates.append((new_seq, new_log_prob))
        
        # Keep only top beam_width candidates based on log probability
        next_candidates.sort(key=lambda x: x[1], reverse=True)
        candidates = next_candidates[:beam_width]
    
    # Return best candidate
    best_seq, _ = candidates[0]
    
    # Convert IDs back to n-grams
    generated_ngrams = []
    for id in best_seq:
        ngram = tokenizer.index_word.get(int(id), 'unk')
        generated_ngrams.append(ngram)
    
    # Join to form text
    generated_text = ''.join(generated_ngrams)
    return generated_text

# Generate dengan beam search
seed = "once upon a time"
generated = generate_text_beam_search(model, seed, length=200, beam_width=3)
print("Generated with beam search:")
print(generated)
```

**Penjelasan**: Beam search track multiple candidate sequences simultaneously. Di setiap step, kita expand top beam_width candidates menjadi top beam_width options, keeping total beam_width sequences. Ini menghasilkan more coherent text karena explore multiple paths instead of greedy single path.

---

## Kesimpulan

Bab ini mengcover perjalanan lengkap language modeling:
1. Download dan preprocess data cerita anak-anak
2. Reduce vocabulary dengan n-grams (dari 14K menjadi 735)
3. Build GRU-based language model
4. Evaluate dengan perplexity metric
5. Generate text dengan greedy decoding dan beam search

GRU dipilih daripada LSTM karena simpler dan lebih efficient, sementara tetap maintain comparable performance. N-grams approach membuat model jauh lebih praktis dengan vocabulary size yang drastis dikurangi. Beam search menghasilkan text quality yang significantly lebih baik dibanding greedy decoding.

Model language modeling ini bisa digunakan sebagai preprocessing untuk task downstream lainnya, atau sebagai foundation untuk building text generation systems seperti story generator atau chatbot.
